In [ ]:
import io, os
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

from tagging.sugest import sugerir_rotulos
from etl.extract import extract_data

In [2]:
contas = ["Conta Corrente XP", "Cartão de Crédito XP"]
labels = ["Data","Valor","Destino / Origem","Descricao","Tipo","Categoria","Subcategoria","Nome","Conta","Notas"]
entry_file = "entry.csv"
backup_folder = "data/backup"
db_file = "data/AllData.csv"

In [3]:
def get_upload_widget() -> widgets.FileUpload:
    return widgets.FileUpload(
        accept=".csv",
        multiple=False,
        description="📁 Selecionar CSV",
        style={"font_weight": "bold"},
        layout=widgets.Layout(padding="8px 15px", width="auto"),
    )


def criar_campos_input():
    conta = widgets.Dropdown(options=contas, layout=widgets.Layout(width='30%'))
    tipo = widgets.Text(description="Tipo:", placeholder="Digite o tipo aqui", layout=widgets.Layout(width='30%'))
    categoria = widgets.Text(description="Categoria:", placeholder="Digite a categoria aqui", layout=widgets.Layout(width='30%'))
    subcategoria = widgets.Text(description="Subcategoria:", placeholder="Digite a subcategoria aqui", layout=widgets.Layout(width='30%'))
    nome = widgets.Text(description="Nome:", placeholder="Digite o nome aqui", layout=widgets.Layout(width='100%'))
    notas = widgets.Textarea(description="Notas:", placeholder="Digite notas aqui", layout=widgets.Layout(width='100%', height='100px'))
    return conta, tipo, categoria, subcategoria, nome, notas

def mostrar_item(row, index, size):
    clear_output(wait=True)
    texto = f"### Item {index+1} / {size}\n"
    for k, v in row.items():
        texto += f"**{k}**: {v}  \n"
    display(Markdown(texto))


In [4]:
def save_row(row):
    if not os.path.exists(entry_file):
        with open(entry_file, "w", encoding="utf-8-sig") as f:
            f.write(','.join(labels) + '\n')

    with open(entry_file, "a", encoding="utf-8-sig") as f:
        f.write(','.join([str(row.get(label, '')) for label in labels]) + '\n')

In [5]:
def handle_upload(_):
    if value := upload_widget.value:
        file_info = next(iter(value.values()))
        content = file_info["content"]
        df = extract_data(io.BytesIO(content))
        process_row(df, 0)


def exibir_upload_widget():
    global upload_widget
    upload_widget = get_upload_widget()
    upload_widget.observe(handle_upload, names="value")
    display(upload_widget)


def process_row(df, index):
    row = df.iloc[index]
    mostrar_item(row, index, len(df))

    conta, tipo, categoria, subcategoria, nome, notas = criar_campos_input()

    if rotulos := sugerir_rotulos(row["Data"], row["Valor"], row["Destino / Origem"], row["Descricao"]):
        tipo.value = rotulos.get("Tipo", "")
        categoria.value = rotulos.get("Categoria", "")
        subcategoria.value = rotulos.get("Subcategoria", "")
        nome.value = rotulos.get("Nome", "")

    linha_superior = widgets.HBox([conta, tipo, categoria, subcategoria], layout=widgets.Layout(gap='10px'))
    linha_media = widgets.HBox([nome])
    linha_inferior = widgets.HBox([notas], layout=widgets.Layout(width='100%'))

    formulario = widgets.VBox([linha_superior, linha_media, linha_inferior], layout=widgets.Layout(width='100%', gap='10px'))

    button = widgets.Button(description="Confirmar")
    output = widgets.Output()

    def on_confirm(_):

        tipo_value = tipo.value.strip()
        categoria_value = categoria.value.strip()
        subcategoria_value= subcategoria.value.strip()
        nome_value = nome.value.strip()

        if not tipo_value:
            with output:
                clear_output()
                print("Por favor, insira um tipo antes de confirmar.")
            return

        if not categoria_value:
            with output:
                clear_output()
                print("Por favor, insira uma categoria antes de confirmar.")
            return
        
        if not subcategoria_value:
            with output:
                clear_output()
                print("Por favor, insira uma subcategoria antes de confirmar.")
            return
        
        if not nome_value:
            with output:
                clear_output()
                print("Por favor, insira um nome antes de confirmar.")
            return
        
        
        row_data = {
            "Data": row["Data"],
            "Valor": row["Valor"],
            "Destino / Origem": row["Destino / Origem"],
            "Descricao": row["Descricao"],
            "Tipo": tipo_value,
            "Categoria": categoria_value,
            "Subcategoria": subcategoria_value,
            "Nome": f'"{nome_value}"',
            "Conta": contas.index(conta.value)+1 if conta.value in contas else conta.value,
            "Notas": notas.value.strip()
        }
        save_row(row_data)
        if index + 1 < len(df):
            process_row(df, index + 1)
        else:
            clear_output()
            exibir_upload_widget()

    button.on_click(on_confirm)
    display(formulario, button, output)


In [6]:
upload_widget = get_upload_widget()
upload_widget.observe(handle_upload, names="value")
display(upload_widget)

### Item 2 / 11
**Data**: 2024-11-07 00:00:00  
**Valor**: -12.92  
**Destino / Origem**: UBER* PENDING  
**Descricao**: Compra no Cartão de Crédito  
**Tipo**: Gasto  


Button(description='Confirmar', style=ButtonStyle())

Output()

In [7]:
from database import FinanceDB
from schemas import FinanceEntrySchema, Account
from tagging.train import train_models
import pandas as pd
from time import time

In [9]:
if os.path.exists(entry_file):
    df = pd.read_csv(entry_file)
    
    # db = FinanceDB()
    # db.connect()
    for index, row in df.iterrows():
        row["Notas"] = row["Notas"] if not pd.isna(row["Notas"]) else "" 
        entry = FinanceEntrySchema(
            data=row["Data"],
            valor=row["Valor"],
            destino_origem=row["Destino / Origem"],
            descricao=row["Descricao"],
            tipo=row["Tipo"],
            categoria=row["Categoria"],
            subcategoria=row["Subcategoria"],
            nome=row["Nome"],
            conta=row["Conta"],
            notas=row["Notas"],
        )
        row["Nome"] = f'"{row["Nome"]}"'

        with open(db_file, "a", encoding="utf-8-sig") as f:
            f.write(','.join([str(row[label]) for label in labels]) + '\n')
    #     db.insert_financa(entry)
    # db.close()
    os.remove(entry_file)
    df.to_csv(f'{backup_folder}/entries/entry_{int(time())}.csv', index=False, encoding="utf-8-sig")

    df = pd.read_csv(db_file, encoding="utf-8-sig")
    df.to_csv(f'{backup_folder}/all_data/AllData_{int(time())}.csv', index=False, encoding="utf-8-sig")
    train_models(db_file)

    

2025-07-14 15:07:32.863 | INFO     | tagging.train:treinar_modelo:51 - Modelo salvo em: tagging/models/modelo_tipo.pkl
2025-07-14 15:07:33.232 | INFO     | tagging.train:treinar_modelo:51 - Modelo salvo em: tagging/models/modelo_categoria.pkl
2025-07-14 15:07:33.629 | INFO     | tagging.train:treinar_modelo:51 - Modelo salvo em: tagging/models/modelo_subcategoria.pkl
2025-07-14 15:07:34.090 | INFO     | tagging.train:treinar_modelo:51 - Modelo salvo em: tagging/models/modelo_nome.pkl
